# Investigation of the reliability of borrowers

Based on the order of the Customer (the bank's credit department), it is necessary to understand whether the marital status and the number of children of the client affect the fact of repayment of the loan on time. Input data from the bank — statistics on the solvency of customers.

The results of the study will be taken into account when building a model of **credit scoring** — a special system that evaluates the ability of a potential borrower to repay a loan to the bank.

**The purpose of the analysis**

The purpose of the study is to answer the following questions:

1) Is there a relationship between having children and repayment of the loan on time?

2) Is there a relationship between marital status and repayment of the loan on time?

3) Is there a relationship between the level of income and repayment of the loan on time?

4) How do the different purposes of the loan affect its repayment on time?

***Table of contents:***

[1 Data overview](#chapter1)

[2 Data preprocessing](#chapter2)

[3 Identification of dependencies for different categories of borrowers](#chapter3)

[Final conclusion](#chapter4)


## 1. Data overview

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
import pprint

data = pd.read_csv('data.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Column names:**

children — the number of children in the family

days_employed — total work experience in days

dob_years — client's age in years

education — the level of education of the client

education_id — identifier of the level of education

family_status — marital status

family_status_id — id of marital status

gender — the gender of the client

income_type — type of employment

debt — whether there was a debt on repayment of loans

total_income — monthly income

purpose — the purpose of obtaining a loan

Let's check for unique values in the columns and their number (if necessary, their share of the total number)

In [3]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

The "-1" value needs to be processed in the 'children' column

In [4]:
data.loc[data['children'] == -1 ].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,-4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,-902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,-3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,349987.852217,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,NaN,57,Среднее,1,женат / замужем,0,F,пенсионер,0,NaN,на покупку своего автомобиля
1363,-1,-1195.264956,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550.699692,профильное образование
1929,-1,-1461.303336,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121.569013,покупка жилья
2073,-1,-2539.761232,42,среднее,1,в разводе,3,F,компаньон,0,162638.609373,покупка жилья
3814,-1,-3045.290443,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы
4201,-1,-901.101738,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375.766751,операции со своей недвижимостью


In [5]:
round((data['children'] == -1).mean(), 3)

0.002

In [6]:
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [7]:
print(data.loc[data['dob_years'] == 0]['dob_years'].count() / data['dob_years'].count() * 100) 

0.4692218350754936


In [8]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

The 'education' column needs to be lowercased.

In [9]:
data['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [10]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

The 'family_status' column needs to be lowercased. 

In [11]:
data['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [12]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [13]:
data.loc[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


In [14]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [15]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

**Intermediate conclusion**

According to the results of the preliminary examination of the data, the following should be noted:

1) Column names do not require changes, however, it is required to fill in the gaps in the columns **days_employed** and **total_income**. Since their number is the same, it is possible that they are not random;

2) In general, the data types in the columns do not require changes for further analysis, with the exception of **day_employed and dob_years**, since the total length of service is measured in days, and the age of the borrower in full years (it is necessary to replace with the int type);

3) In the column **children** there are 47 rows with the value "-1", which is 0.2% of the total. Since their number is insignificant, and it is not possible to find out the exact reasons for this filling (perhaps there was a child's death), it is proposed to delete these lines in the data preprocessing. In addition, there are 76 rows in the specified column with the value "20" (artifact).It is not possible to find out the exact reasons for this filling, however, given the proximity of the keys 2 and 0 on standard user keyboards, the absence of any intermediate values between 5 and 20 children, it is assumed that filling with the value "2" was meant.;

4) There are incorrect values (artifacts) in the column **days_employed**;

5) In the column **dob_years** there are 101 rows with "0" age of customers, which is impossible(0.5% of the total number). In the preprocessing of the data, it is proposed to group them into age groups depending on the type of employment and replace the value "0" with the average value / median (based on the difference between them).;

6) In the column **education**, **family_status** contains a large number of repetitions due to the use of different registers in the values. In the process of preprocessing, it should be brought to the lower case;

7) In the column **gender** there is an 'XNA' value that cannot be attributed to either male(M) or female(W) in the Russian Federation. Since this value is only 1, it is suggested to delete it. In addition, the values in the column are set out in Latin, in the initial columns in Russian. As part of the preprocessing, it is proposed to replace with "M" and "W", respectively;

8) In the column **income_type**, you should pay attention to the two values "companion" and "on maternity leave". Companion is a profession that involves caring for people on a paid basis (close to the profession - a nurse). As a rule, representatives of this profession are employees of a specialized agency or entrepreneurs. "On maternity leave" - the status of a temporarily unemployed co-worker/ employee of the organization. Since both categories are actually employees, it is proposed to assign them to this category.

The identified comments will be processed in the "Data Preprocessing" section

## 2. Data preprocessing

First of all, we will eliminate the comments established during Data overview stage.

In [16]:
# Delete the rows with the value "-1" in the children column.

data.drop(data[data['children'] == -1].index, inplace = True)

data['children'] = data['children'].replace(20, 2)

In [17]:
# Checking the removal and replacement of 20 by 2

data['children'].value_counts()

0    14149
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [18]:
# Lowercase 'education' column values

data['education'] = data['education'].str.lower()

data['family_status'] = data['family_status'].str.lower()

In [19]:
data['education'].value_counts()

среднее                15196
высшее                  5251
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

In [20]:
data['family_status'].value_counts()

женат / замужем          12351
гражданский брак          4172
не женат / не замужем     2808
в разводе                 1191
вдовец / вдова             956
Name: family_status, dtype: int64

In [21]:
# deleting the row with the value XNA in the gender column and replacing the Latin name of the gender with Russian

data.drop(data[data['gender'] == 'XNA'].index, inplace=True)

data['gender'] = data['gender'].replace('F', 'Ж')
data['gender'] = data['gender'].replace('M', 'М')

In [22]:
# Checking for deletion and replacement

data['gender'].value_counts()

Ж    14201
М     7276
Name: gender, dtype: int64

In [23]:
# Changing "компаньона" and "в декрете" to employee in the 'income_type' column

data['income_type'] = data['income_type'].replace('компаньон', 'сотрудник')

data['income_type'] = data['income_type'].replace('в декрете','сотрудник')


In [24]:
# Checking for replacement

data['income_type'].value_counts()

сотрудник          16169
пенсионер           3848
госслужащий         1455
предприниматель        2
безработный            2
студент                1
Name: income_type, dtype: int64

Since it was decided to use the average/median value for the age group of borrowers, depending on the type of employment, to replace the zero values in the **dog_years** column, we will group the data based on the specified parameters.

In [25]:
# Let's select rows with zero values in the dob_years column in a separate dictionary by the type of employment
income_type_dict = data[data['dob_years'] == 0]['income_type']
income_type_dict = income_type_dict.drop_duplicates().reset_index(drop = True)
income_type_dict


0      пенсионер
1      сотрудник
2    госслужащий
Name: income_type, dtype: object

Replace the zero values with the median value for each category of borrowers, depending on the type of employment.
Since the difference between the mean and median is insignificant (the values are generally evenly distributed), the choice between the latter should not be of significant importance for the results of the study. In addition, since full years were recorded in the data, the final values should be converted to the int type.

In [26]:
data['dob_years'].describe()

count    21477.000000
mean        43.295851
std         12.576916
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [27]:
data['dob_years'] = data.groupby('income_type')['dob_years'].transform(lambda group : group.fillna(group.median()))

In [28]:
print(data['dob_years'].mean())
print(data['dob_years'].median())
    
data['dob_years'] = data['dob_years'].astype('int64')

43.295851375890486
42.0


In [29]:
# Checking the filling of zero values in the dob_years column

data['dob_years'].value_counts().sort_index().head(10)

0     101
19     14
20     51
21    111
22    183
23    253
24    263
25    357
26    407
27    492
Name: dob_years, dtype: int64

Based on the materials of the article by V.A. Tsygankov and S.L.Zharkov "Classification and systematization of the able-bodied population by age groups", the division of borrowers can be carried out using the function into the following categories:

In [30]:
def borrower_age_group(age):
    if age < 18:
        return 'несовершеннолетний'
    if age >= 18 and  age <= 24:
        return 'молодой'
    elif age > 25 and age <= 44:
        return 'зрелый'
    elif age >= 45 and age <= 64:
        return 'пожилой'
    else:
        return 'старый'

data['borrower_age_group'] = data['dob_years'].apply(borrower_age_group) 

In [31]:
# Let's separate the values of age groups from the borrower_age_group column into a separate dictionary to further eliminate gaps in columns depending on the type of employment and age group
borrower_age_dict = data['borrower_age_group']
borrower_age_dict = borrower_age_dict.drop_duplicates().reset_index(drop = True)
borrower_age_dict

0                зрелый
1               пожилой
2                старый
3               молодой
4    несовершеннолетний
Name: borrower_age_group, dtype: object

### Processing of missing values

Let's check the data for empty values:

In [32]:
print(data.isna().sum())

children                 0
days_employed         2171
dob_years                0
education                0
education_id             0
family_status            0
family_status_id         0
gender                   0
income_type              0
debt                     0
total_income          2171
purpose                  0
borrower_age_group       0
dtype: int64


Since the number of empty values in the columns **days_employed** and **total_income** is the same, it can be assumed that their occurrence is due to the procedure of data deletion or possible merging of several tables. Let's check if there are differences depending on the grouping of **days_employed** and **total_income** with different columns in order to develop a common approach for replacing of missing values in both columns

In [33]:
data['days_employed'].isna().groupby(data['income_type']).sum().astype('int64')

income_type
безработный           0
госслужащий         147
пенсионер           412
предприниматель       1
сотрудник          1611
студент               0
Name: days_employed, dtype: int64

In [34]:
data['total_income'].isna().groupby(data['income_type']).sum().astype('int64')

income_type
безработный           0
госслужащий         147
пенсионер           412
предприниматель       1
сотрудник          1611
студент               0
Name: total_income, dtype: int64

1) Since the number of missing values in the groups by time of service and total income, depending on the type of employment, is the same, it is proposed to fill them with median values in the corresponding group, depending on the type of employment;

2) additionally analyze the line by type of employment entrepreneur with a missing value

In [35]:
data[data['income_type'] == 'предприниматель']['income_type'].count()

2

In [36]:
data[(data['days_employed'].isna())&(data['income_type'] == 'предприниматель')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,borrower_age_group
5936,0,NaN,58,высшее,0,женат / замужем,0,М,предприниматель,0,NaN,покупка жилой недвижимости,пожилой


Since there are only 2 rows with the "entrepreneur" parameter in **income_type**, in one of which there are no values in the columns **days_employed** and **total_income**, it is not possible to fill in the gap with an average/ median value, therefore it is proposed to exclude this row from the study.

In [37]:
# Deleting a line with a missing value

data.drop(data[(data['days_employed'].isna())&(data['income_type'] == 'предприниматель')].index, inplace=True)

In [38]:
# checking the deletion

data[data['income_type'] == 'предприниматель']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,borrower_age_group
18697,0,-520.848083,27,высшее,0,гражданский брак,1,Ж,предприниматель,0,499163.144947,на проведение свадьбы,зрелый


Let's replace the gaps in the column **days_employed** with median values depending on the type of employment

In [39]:
data['days_employed'] = data.groupby('income_type')['days_employed'].transform(lambda group : group.fillna(group.median()))

Заменим пропуски в колонке **total_income** медианными значениями  в зависимости от типа занятости

In [40]:
# Let's check the values of the total_income column for extreme values, as well as the absence of negative ones, and the uniformity of the distribution

data['total_income'].describe()

count    1.930600e+04
mean     1.674513e+05
std      1.030369e+05
min      2.066726e+04
25%      1.030109e+05
50%      1.450117e+05
75%      2.034498e+05
max      2.265604e+06
Name: total_income, dtype: float64

In the values of the column **total_income** there are no negative values, artifacts, therefore, it is possible to fill in empty fields with median values depending on the age and type of employment of borrowers  

In [41]:
# let's create an auxiliary dictionary containing unique types of employment with missing values of the total_income column 
# let's remove duplicates

income_type_dict = data[data['total_income'].isna()]['income_type']
income_type_dict = income_type_dict.drop_duplicates().reset_index(drop=True)

# let's create a loop that replaces all empty values with the median of the corresponding group by type of employment and age
for income_type in income_type_dict:
    for age_group in borrower_age_dict:
        group_median = data[(data['income_type'] == income_type)&(data['total_income'].isna() == False)&(data['borrower_age_group'] == age_group)]['total_income'].median()
        data.loc[(data['total_income'].isna()==True)&(data['income_type']==income_type)&(data['borrower_age_group'] == age_group),'total_income'] = group_median 


Let's check the replacement of the gaps in the columns **total_income** and **days_employed**

In [42]:
data.isna().sum()

children              0
days_employed         0
dob_years             0
education             0
education_id          0
family_status         0
family_status_id      0
gender                0
income_type           0
debt                  0
total_income          0
purpose               0
borrower_age_group    0
dtype: int64

**Intermediate conclusion**

The replacement of the missing valuesd was carried out successfully. Empty values have been replaced with median values depending on the borrower's group by type of employment and age. The exact reasons for the formation of omissions are unknown. One of the possible ones may be incorrect data collection or loss of values when merging tables

### Replacement of the data type

The initial data set has a type that does not interfere with the conduct of the analysis. In the course of eliminating the problems identified during the preliminary examination of the data, the data type was replaced from float to int for the age of the borrower, income. However, the data in the column **days_employed** should be also taken into account, since it is necessary to eliminate negative values (take days modulo) and bring them to an integer type. In addition, since the values in the column are indicated in days, it is necessary to check for the presence of artifacts, taking into account their division by years, depending on the type of employment of borrowers.

In [43]:
data['days_employed'].describe()

count     21476.000000
mean      63566.551429
std      141167.750253
min      -18388.949901
25%       -2569.254663
50%       -1355.448720
75%        -315.693544
max      401755.400475
Name: days_employed, dtype: float64

In [44]:
# let's find the absolute integer average of the length of service of borrowers depending on the type of employment (in years)

((data.groupby('income_type')['days_employed'].mean()).abs()/365).astype('int64')

income_type
безработный        1003
госслужащий           9
пенсионер          1000
предприниматель       1
сотрудник             5
студент               1
Name: days_employed, dtype: int64

From the absolute average values of the time of service (in years), depending on the type of employment, the indicators of the group of pensioners and unemployed are distinguished (the time of service cannot be equal to 1000 years). Based on the fact that with the legislation of the Russian Federation in force for 2021, the retirement age for men is 65 years, and for women - 60, it is proposed to replace the seniority index of pensioners by the product of 42 and 47 years (according to the general rule, 18 years is the age of the beginning of full-fledged work), depending on gender, by 365 days. The artifact in the group of unemployed requires additional study. In addition, it is required to replace all the values of the time of service with their absolute values.

In [45]:
# Let's check the number of values in the unemployed group

data['income_type'].value_counts()

сотрудник          16169
пенсионер           3848
госслужащий         1455
безработный            2
студент                1
предприниматель        1
Name: income_type, dtype: int64

In [46]:
# we will display artifacts in the group of unemployed

data.loc[data['income_type'] == 'безработный']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,borrower_age_group
3133,1,337524.466835,31,среднее,1,женат / замужем,0,М,безработный,1,59956.991984,покупка жилья для сдачи,зрелый
14798,0,395302.838654,45,высшее,0,гражданский брак,1,Ж,безработный,0,202722.511368,ремонт жилью,пожилой


It is impossible to determine the reason for the appearance of these values from the derived artifacts based on work experience for a group of unemployed. A possible reason may be the indication of the time of service in hours. Due to the fact that there are only 2 of these values, they can either be deleted or replaced by the difference between the age of borrowers and the standard age of starting work (18 years)(let's specify 13.0 and 27.0 years respectively in the fields).

In [47]:
data.loc[data['income_type'] == 'безработный']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,borrower_age_group
3133,1,337524.466835,31,среднее,1,женат / замужем,0,М,безработный,1,59956.991984,покупка жилья для сдачи,зрелый
14798,0,395302.838654,45,высшее,0,гражданский брак,1,Ж,безработный,0,202722.511368,ремонт жилью,пожилой


In [48]:
data.loc[(data['income_type'] == 'безработный') & (data['dob_years'] == 31),'days_employed'] = 13.0 * 365

data.loc[(data['income_type'] == 'безработный') & (data['dob_years'] == 45),'days_employed'] = 27.0 * 365

In [49]:
data.loc[data['income_type'] == 'безработный']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,borrower_age_group
3133,1,4745.0,31,среднее,1,женат / замужем,0,М,безработный,1,59956.991984,покупка жилья для сдачи,зрелый
14798,0,9855.0,45,высшее,0,гражданский брак,1,Ж,безработный,0,202722.511368,ремонт жилью,пожилой


In [50]:
# Replacing the seniority of a group of pensioners, depending on gender, by 42 years for women and 47 for men

data['days_employed'] = data['days_employed'].abs()

In [51]:
# clarification of the seniority of female pensioners

data.loc[(data['income_type'] == 'пенсионер') & (data['gender'] == 'Ж'), 'days_employed'] = 42*365

# clarification of the seniority of male pensioners

data.loc[(data['income_type'] == 'пенсионер') & (data['gender'] == 'М'), 'days_employed'] = 47*365

In [52]:
data['days_employed'] = data['days_employed'].astype('int64')

In [53]:
# let's check the changes and transformations made
data['days_employed'].describe()

count    21476.000000
mean      4680.816213
std       5511.457436
min         24.000000
25%       1025.000000
50%       1991.500000
75%       5347.500000
max      18388.000000
Name: days_employed, dtype: float64

In [54]:
# let's check the absolute integer averages of the time of service of borrowers depending on the type of employment (in years) 

((data.groupby('income_type')['days_employed'].mean())/365).astype('int64')

income_type
безработный        20
госслужащий         9
пенсионер          42
предприниматель     1
сотрудник           5
студент             1
Name: days_employed, dtype: int64

In [55]:
data.dtypes

children                int64
days_employed           int64
dob_years               int64
education              object
education_id            int64
family_status          object
family_status_id        int64
gender                 object
income_type            object
debt                    int64
total_income          float64
purpose                object
borrower_age_group     object
dtype: object

**Intermediate conclusion**

In the course of the transformations performed, artifacts in the borrowers' work experience were eliminated, as well as conversion to an integer data type was performed. The selected data types do not prevent further analysis.

### Duplicates processing

In [56]:
# Let's check for duplicates in the data

data.duplicated().sum()

95

In [57]:
# let's print the first 5 duplicate values

data[data.duplicated() == True].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,borrower_age_group
2606,0,1568,21,среднее,1,не женат / не замужем,4,М,сотрудник,0,128479.293598,операции с коммерческой недвижимостью,молодой
2849,0,1568,41,среднее,1,женат / замужем,0,Ж,сотрудник,0,153481.209137,покупка жилья для семьи,зрелый
3290,0,15330,58,среднее,1,гражданский брак,1,Ж,пенсионер,0,121685.132557,сыграть свадьбу,пожилой
4182,1,1568,34,высшее,0,гражданский брак,1,Ж,сотрудник,0,153481.209137,свадьба,зрелый
4851,0,15330,60,среднее,1,гражданский брак,1,Ж,пенсионер,0,121685.132557,свадьба,пожилой


In [58]:
# Let's delete duplicates without saving old indexes

data = data.drop_duplicates().reset_index(drop=True)

In [59]:
# Let's check the removal of duplicates

data.duplicated().sum()

0

**Intermediate conclusion**

All the identified duplicates have been deleted.

### Lemmatization

For further research and an answer to the tasks set in the project, it is necessary to categorize the data provided. Previously, we used groupings by age and type of employment to fill in the gaps. However, in order to further assess the return of loans, we can use categorization by loan goals and the level of borrowers' income (an inetere approach is used in the article by N.E. Tikhonova, Yu.V. Lezhnina "Models of stratification of Russian society by income: approaches and results" on page 197 on the median division of the sample). In this part, we will highlight the main objectives of loans.

In [60]:
# We will carry out a preliminary assessment of the loan goals by highlighting the lemmas of the goals, counting the values and then forming a single dictionary of goals with the most common values. 

lemmas_dict = []

# let's define all the unique values in the purpose column

unique_purpose_dict = data['purpose'].unique()

unique_purpose_dict

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [61]:
# Лемматизация целей займов
m = Mystem()
purpose = ';'.join(unique_purpose_dict)
pprint.pprint(purpose)

('покупка жилья;приобретение автомобиля;дополнительное образование;сыграть '
 'свадьбу;операции с жильем;образование;на проведение свадьбы;покупка жилья '
 'для семьи;покупка недвижимости;покупка коммерческой недвижимости;покупка '
 'жилой недвижимости;строительство собственной '
 'недвижимости;недвижимость;строительство недвижимости;на покупку подержанного '
 'автомобиля;на покупку своего автомобиля;операции с коммерческой '
 'недвижимостью;строительство жилой недвижимости;жилье;операции со своей '
 'недвижимостью;автомобили;заняться образованием;сделка с подержанным '
 'автомобилем;получение образования;автомобиль;свадьба;получение '
 'дополнительного образования;покупка своего жилья;операции с '
 'недвижимостью;получение высшего образования;свой автомобиль;сделка с '
 'автомобилем;профильное образование;высшее образование;покупка жилья для '
 'сдачи;на покупку автомобиля;ремонт жилью;заняться высшим образованием')


In [62]:
lemmas = m.lemmatize(purpose)
pprint.pprint(Counter(lemmas))

Counter({' ': 59,
         ';': 37,
         'покупка': 10,
         'недвижимость': 10,
         'автомобиль': 9,
         'образование': 9,
         'жилье': 7,
         'с': 5,
         'операция': 4,
         'на': 4,
         'свой': 4,
         'свадьба': 3,
         'строительство': 3,
         'получение': 3,
         'высокий': 3,
         'дополнительный': 2,
         'для': 2,
         'коммерческий': 2,
         'жилой': 2,
         'подержать': 2,
         'заниматься': 2,
         'сделка': 2,
         'приобретение': 1,
         'сыграть': 1,
         'проведение': 1,
         'семья': 1,
         'собственный': 1,
         'со': 1,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})


**Intermediate conclusion**

Analyzing the lemmas highlighted in this section, we can identify the following main purposes for which loans are taken:

1) real estate

2) education

3) wedding

4) car

### Categorization of data

In this section, it is proposed to allocate the following categories of borrowers:

1) depending on the purpose of the loan (based on the lemmas from the previous section, we will form a dictionary of the main goals of the loan and carry out a categorization with the entry of values in a new column **purpose_type**)

2) depending on the borrower's income level(The main idea is to estimate the income level of the corresponding borrower with a median value in the sample, so if the income is below half of the median value, then it is considered "very low", in the range of 0.5 median < "low"<= 0.75 median, 0.75 median < "average" <= 1.25 median, 1.25 median < "higher average" <= 2 medians, 2 medians < "high" <= 4 medians, anything higher is considered "very high"). Based on the results of such categorization, it is proposed to introduce an additional column with the specified categories **total_income_type**.

In [63]:
# let's form a dictionary for further categorization with an indication of the category selected on the basis of the selected lemmas under the index 0lemmas of the main goals     

lemmas_dict = [['недвижимость', 'покупка жилья','операции с жильем', 'покупка жилья для семьи',  'покупка недвижимости', 'покупка коммерческой недвижимости', 'покупка жилой недвижимости','строительство собственной недвижимости', 'недвижимость', 'строительство недвижимости', 'операции с коммерческой недвижимостью','строительство жилой недвижимости', 'жилье', 'операции со своей недвижимостью','покупка жилья для сдачи','покупка своего жилья', 'операции с недвижимостью','профильное образование','ремонт жилью'],
               ['автомобиль','приобретение автомобиля', 'на покупку подержанного автомобиля','на покупку своего автомобиля','автомобили', 'сделка с подержанным автомобилем', 'свой автомобиль', 'сделка с автомобилем','на покупку автомобиля'],
               ['свадьба', 'сыграть свадьбу','на проведение свадьбы'],
               ['образование', 'дополнительное образование', 'заняться образованием', 'заняться высшим образованием', 'получение образования', 'получение дополнительного образования','получение высшего образования','высшее образование']]


In [64]:
# let's write a goal categorization function depending on the selected lemmas and fill in a new column purpose_type
def purpose_type_group(purpose):
    
    for lemma in lemmas:
        for l_dict in lemmas_dict:
            if lemma in l_dict:
                return l_dict[0]
            

data['purpose_type'] = data['purpose'].apply(purpose_type_group)

In [65]:
# Let's check the categorization made

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,borrower_age_group,purpose_type
0,1,8437,42,высшее,0,женат / замужем,0,Ж,сотрудник,0,253875.639453,покупка жилья,зрелый,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,Ж,сотрудник,0,112080.014102,приобретение автомобиля,зрелый,недвижимость
2,0,5623,33,среднее,1,женат / замужем,0,М,сотрудник,0,145885.952297,покупка жилья,зрелый,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,М,сотрудник,0,267628.550329,дополнительное образование,зрелый,недвижимость
4,0,15330,53,среднее,1,гражданский брак,1,Ж,пенсионер,0,158616.077870,сыграть свадьбу,пожилой,недвижимость


In [66]:
# Categorization based on income level and filling in the additional column total_income_group
total_income_median = data['total_income'].median()


def total_income_group(total_income):
    if total_income <= total_income_median*0.5:
        return 'очень низкий'
    elif total_income > total_income_median*0.5 and total_income <= total_income_median*0.75:
        return 'низкий'
    elif total_income > total_income_median*0.75 and total_income <= total_income_median*1.25:
        return 'средний'
    elif total_income > total_income_median*1.25 and total_income <= total_income_median*2:
        return 'выше среднего'
    elif total_income > total_income_median*2 and total_income <= total_income_median*4:
        return 'высокий'
    else:
        return 'очень высокий'
    
data['total_income_group'] = data['total_income'].apply(total_income_group)    


In [67]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,borrower_age_group,purpose_type,total_income_group
0,1,8437,42,высшее,0,женат / замужем,0,Ж,сотрудник,0,253875.639453,покупка жилья,зрелый,недвижимость,выше среднего
1,1,4024,36,среднее,1,женат / замужем,0,Ж,сотрудник,0,112080.014102,приобретение автомобиля,зрелый,недвижимость,низкий
2,0,5623,33,среднее,1,женат / замужем,0,М,сотрудник,0,145885.952297,покупка жилья,зрелый,недвижимость,средний
3,3,4124,32,среднее,1,женат / замужем,0,М,сотрудник,0,267628.550329,дополнительное образование,зрелый,недвижимость,выше среднего
4,0,15330,53,среднее,1,гражданский брак,1,Ж,пенсионер,0,158616.077870,сыграть свадьбу,пожилой,недвижимость,средний


In [68]:
data['purpose_type'].value_counts()

недвижимость    21381
Name: purpose_type, dtype: int64

In [69]:
data['total_income_group'].value_counts()

средний          9422
выше среднего    4446
низкий           4180
очень низкий     1849
высокий          1375
очень высокий     109
Name: total_income_group, dtype: int64

In [70]:
# altornative approach by std 

data.groupby('total_income_group')['total_income'].std()

total_income_group
высокий           69521.009282
выше среднего     30991.954433
низкий            10699.074184
очень высокий    322540.756579
очень низкий      11285.512330
средний           19341.686897
Name: total_income, dtype: float64

In [71]:
# Let's build a summary table to analyze the results of categorization, taking into account the types of employment

data.pivot_table(index=['income_type'], columns='total_income_group', values='total_income', aggfunc='count')

total_income_group,высокий,выше среднего,низкий,очень высокий,очень низкий,средний
income_type,,,,,,
безработный,NaN,1.0,NaN,NaN,1.0,NaN
госслужащий,109.0,306.0,267.0,7.0,117.0,647.0
пенсионер,121.0,548.0,973.0,7.0,666.0,1506.0
предприниматель,1.0,NaN,NaN,NaN,NaN,NaN
сотрудник,1144.0,3591.0,2939.0,95.0,1065.0,7269.0
студент,NaN,NaN,1.0,NaN,NaN,NaN


In [72]:
data.pivot_table(index=['income_type'], columns='purpose_type', values='total_income', aggfunc='count')

purpose_type,недвижимость
income_type,
безработный,2
госслужащий,1453
пенсионер,3821
предприниматель,1
сотрудник,16103
студент,1


**Intermediate counclusion**

From the data of the pivot tables , you can see:

1) regardless of the type of employment, the main loans are related to the purchase of housing, then a car;

2) high uncertainty and associated risks exist for the category of borrowers "unemployed", since there are two extreme values with very high and low income. Presumably, the first refers to a person who is actually engaged in entrepreneurial activity, but the absence of a direct indication of this does not allow him to qualify as a reliable borrower. The second person is most likely really unemployed and receives benefits.

3) the category of borrowers "pensioners" is noteworthy, since in addition to employees, about a little more than half of pensioners have an average income and higher. Perhaps this is due to obtaining additional income in addition to a pension and continuing to work. This category can be considered a potential growth point. Perhaps it is necessary to develop separate products for this category for the profile department of the bank. However, to answer this question, it is necessary to conduct an additional assessment of the loan repayment from the type of employment.

## Identification of dependencies for different categories of borrowers

**The relationship between having children and repayment of the loan on time**

In [73]:
# Let's calculate the proportion of borrowers who have overdue loan repayments, depending on the number of children
# Average value for each category

print(data.groupby('children')['debt'].mean().sort_values(ascending=True))

children
5    0.000000
0    0.075545
3    0.081818
1    0.092404
2    0.095059
4    0.097561
Name: debt, dtype: float64


In [74]:
# et's calculate the proportion of borrowers who have overdue debt repayment, depending on the number of children

data.pivot_table(index=['children'], columns='income_type', values='debt', aggfunc='mean')

income_type,безработный,госслужащий,пенсионер,предприниматель,сотрудник,студент
children,,,,,,
0,0.0,0.068129,0.056410,0.0,0.083161,0.0
1,1.0,0.053672,0.050909,NaN,0.098204,NaN
2,NaN,0.031746,0.103448,NaN,0.101206,NaN
3,NaN,0.055556,0.166667,NaN,0.083333,NaN
4,NaN,0.000000,0.000000,NaN,0.121212,NaN
5,NaN,0.000000,NaN,NaN,0.000000,NaN


In [75]:
# As an experiment, I will try to visualize the table above

import seaborn as sns
cm = sns.light_palette("blue", as_cmap=True)

round(data.pivot_table(index=['children'], columns='income_type', values='debt', aggfunc='mean'), 3).style.background_gradient(cmap=cm)

income_type,безработный,госслужащий,пенсионер,предприниматель,сотрудник,студент
children,,,,,,
0,0.000000,0.068000,0.056000,0.000000,0.083000,0.000000
1,1.000000,0.054000,0.051000,nan,0.098000,nan
2,nan,0.032000,0.103000,nan,0.101000,nan
3,nan,0.056000,0.167000,nan,0.083000,nan
4,nan,0.000000,0.000000,nan,0.121000,nan
5,nan,0.000000,nan,nan,0.000000,nan


**Intermediate conclusion**

The most disciplined borrowers are persons without children. The least - persons with 4 children. It is important to note that the number of values with borrowers with 5 children is not enough to qualify them as reliable. For civil servants, the fact of repayment of the loan on time in general does not depend much on the number of children, the leading category is civil servants with 2 children. Pensioners and employees have the tendecy of an increase in loan debt with an increase in the number of children


**The relationship between marital status and repayment of the loan on time**

In [76]:
# Let's calculate the proportion of borrowers who have overdue loan repayments, depending on marital status
# Average value for each category

print(data.groupby('family_status')['debt'].mean().sort_values(ascending=True))

family_status
вдовец / вдова           0.065969
в разводе                0.071429
женат / замужем          0.075665
гражданский брак         0.093697
не женат / не замужем    0.097718
Name: debt, dtype: float64


In [77]:
# let's calculate the proportion of borrowers who have overdue debt repayment, depending on marital status and type of employment

data.pivot_table(index=['family_status'], columns='income_type', values='debt', aggfunc='mean')

income_type,безработный,госслужащий,пенсионер,предприниматель,сотрудник,студент
family_status,,,,,,
в разводе,NaN,0.049383,0.059633,NaN,0.076319,NaN
вдовец / вдова,NaN,0.044444,0.069159,NaN,0.064000,NaN
гражданский брак,0.0,0.068966,0.055470,0.0,0.103438,NaN
женат / замужем,1.0,0.055432,0.055072,NaN,0.082099,NaN
не женат / не замужем,NaN,0.073171,0.045845,NaN,0.107424,0.0


**Intermediate conclusion**

Based on the above results, it can generally be attributed to the group of reliable borrowers: divorced persons who are officially married, widowers and widows. However, it is necessary to take into account the type of employment of the borrower. The group of unmarried persons, with the exception of pensioners, can be attributed to the most risky.

**The relationship between the income level and the repayment of the loan on time**

In [78]:
# Let's calculate the share of borrowers who have overdue loan repayments, depending on the income level
# Average value for each category

print(data.groupby('total_income_group')['debt'].mean().sort_values(ascending=True))

total_income_group
высокий          0.071273
очень низкий     0.073012
очень высокий    0.073394
выше среднего    0.074674
низкий           0.083014
средний          0.087030
Name: debt, dtype: float64


In [79]:
# let's calculate the proportion of borrowers who have overdue debt repayment, depending on the type of employment and income level

data.pivot_table(index=['total_income_group'], columns='income_type', values='debt', aggfunc='mean')

income_type,безработный,госслужащий,пенсионер,предприниматель,сотрудник,студент
total_income_group,,,,,,
высокий,NaN,0.018349,0.041322,0.0,0.079545,NaN
выше среднего,0.0,0.071895,0.062044,NaN,0.076859,NaN
низкий,NaN,0.063670,0.048304,NaN,0.096291,0.0
очень высокий,NaN,0.000000,0.000000,NaN,0.084211,NaN
очень низкий,1.0,0.085470,0.051051,NaN,0.084507,NaN
средний,NaN,0.054096,0.063745,NaN,0.094786,NaN


In [80]:
# let's calculate the share of borrowers who have overdue debt repayment, depending on the level of income and the number of children

data.pivot_table(index=['total_income_group'], columns='children', values='debt', aggfunc='mean')

children,0,1,2,3,4,5
total_income_group,,,,,,
высокий,0.062428,0.079114,0.106918,0.062500,0.000000,NaN
выше среднего,0.071059,0.087332,0.074592,0.029412,0.250000,0.0
низкий,0.077889,0.093960,0.096552,0.052632,0.142857,0.0
очень высокий,0.098361,0.033333,0.071429,0.000000,NaN,NaN
очень низкий,0.066109,0.088319,0.075949,0.217391,NaN,0.0
средний,0.080241,0.097606,0.105376,0.102740,0.074074,0.0


**Intermediate conclusion**

Borrowers with high and higher income levels are the most reliable borrowers. In addition, they have a pattern of decreasing debt values with an increase in the number of children. It is important to note that the values obtained have a normal distribution, however, two categories are distinguished: "above average" and "low" with 4 children. The debt values here deviate significantly from the average income level line. The reason for the appearance of these values is not yet possible to establish. It is proposed to conduct additional analysis for these categories and, if possible, increase the sample size.

**The impact of the purpose of the loan on its repayment**

In [81]:
# Let's calculate the proportion of borrowers who have overdue loan repayments, depending on the purpose of the loan
# lets's find the average value for each category

print(data.groupby('purpose_type')['debt'].mean().sort_values(ascending=True))

purpose_type
недвижимость    0.081381
Name: debt, dtype: float64


In [82]:
# let's calculate the proportion of borrowers who have overdue debt repayment, depending on the type of employment and the purpose of the loan

data.pivot_table(index=['purpose_type'], columns='income_type', values='debt', aggfunc='mean')

income_type,безработный,госслужащий,пенсионер,предприниматель,сотрудник,студент
purpose_type,,,,,,
недвижимость,0.5,0.059188,0.05653,0.0,0.089238,0.0


In [83]:
# let's calculate the proportion of borrowers who have overdue debt repayment, depending on the number of children and the purpose of the loan

data.pivot_table(index=['purpose_type'], columns='children', values='debt', aggfunc='mean')

children,0,1,2,3,4,5
purpose_type,,,,,,
недвижимость,0.075545,0.092404,0.095059,0.081818,0.097561,0.0


**Intermediate conclusion**

Regardless of the type of employment and the number of borrowers' children, the lowest debt indicators are those who take out a loan for real estate transactions and family (wedding). It is important to note the presence of artifacts for borrowers with 4 children and the purpose of carrying out any real estate transactions, as well as for persons with 3 children and the purpose of a wedding. In order to identify specific goals that affect these indicators, it is necessary to conduct additional analysis for each unique position included in the corresponding category. The most risky objectives of the loan are: education and a car

##  Final conclusion

It can be noted that the number of children of the borrower affects the timely repayment of the loan ambiguously (depending on the purpose of the loan, the type of employment and income level, there are artifacts). In general, the group of reliable borrowers can include: divorced persons who are officially married, widowers and widows. However, it is necessary to take into account the type of employment of the borrower. The group of unmarried persons, with the exception of pensioners, can be attributed to the most risky. When issuing a loan, it is recommended to use comprehensive metrics based on income level, loan purpose and marital status.

It is important to note the presence of artifacts for borrowers with 4 children and the purpose of carrying out any real estate transactions, as well as for persons with 3 children and the purpose of a wedding. Borrowers with high and higher income levels are the most reliable borrowers, however, two categories are distinguished by income level: "above average" and "low" with 4 children with a high percentage of non-repayment on time. For this category of borrowers, it is recommended to ask for additional loan collateral.

In the process of preprocessing, non-random anomalies were found: the number of missed lines by time of service and income level. It is necessary to further clarify the reasons for the formation of this anomaly. An anomaly was also revealed for the category of pensioners (overstated seniority indicators). The replacement made in this project with fixed indicators should not affect the results of the study, however, in the case of additional research in this category, incorrect indicators may occur, therefore, specialized specialists are recommended to find out the cause of the anomaly.